In [6]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [10]:
import getpass
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
print(os.environ["LANGCHAIN_API_KEY"])

In [11]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral", temperature=0.1)


In [13]:
llm.invoke("What is langgraph?")

" LangGraph is a large-scale, multilingual graph database designed for natural language processing (NLP) tasks. It was developed by the Facebook AI Research (FAIR) team and open-sourced in 2019. The main goal of LangGraph is to provide a scalable solution for storing and querying large amounts of linguistic data, such as words, phrases, and their relationships, across multiple languages. This can help improve the performance of NLP models by providing them with more contextualized and accurate data.\n\nLangGraph stores its data in a graph format, which allows it to represent complex relationships between entities (e.g., words or phrases) and their attributes (e.g., part-of-speech tags, synonyms, etc.). This makes it easier for NLP models to understand the context and meaning of words and sentences, as well as to make predictions based on that understanding.\n\nLangGraph is built using Apache TinkerPop's Gremlin graph processing language and is designed to be highly scalable, allowing i

In [14]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
  web_path="https://blog.langchain.dev/langgraph/"
)
docs = loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://blog.langchain.dev/langgraph/', 'title': 'LangGraph', 'language': 'en'}, page_content='\n\n\nLangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024\n\n\n\n\n\nTL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical

In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000, 
  chunk_overlap=200, 
  add_start_index=True
)


In [35]:
all_splits = text_splitter.split_documents(docs)

In [44]:
from langchain_community import embeddings

embeddings = embeddings.OllamaEmbeddings(
  model="nomic-embed-text"
)

In [45]:
from langchain_community.vectorstores import Chroma

vector_store = Chroma.from_documents(
  documents=all_splits,
  embedding=embeddings,
)

In [53]:
retriever = vector_store.as_retriever(
  search_kwargs={"k": 6},
  search_type="similarity",
)

In [55]:
retriever.invoke("What is langgraph?")

[Document(metadata={'language': 'en', 'source': 'https://blog.langchain.dev/langgraph/', 'start_index': 3, 'title': 'LangGraph'}, page_content='LangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024'),
 Document(metadata={'language': 'en', 'source': 'https://blog.langchain.dev/langgraph/', 'start_index': 7607, 'title': 'LangGraph'}, page_content='"model",\n    should_continue,\n    {\n        "end": END,\n        "continue": "tools"\n    }\n)CompileAfter we define our graph, we can compile it into a runnable! This simply takes the graph definition we\'ve created so far an returns a runnable. This runnable exposes all the same method as LangChain runnables (.invoke, .stream, .

In [60]:
from langchain import hub

promt = hub.pull("rlm/rag-prompt")

/Users/hansel/Documents/Belajar/python/ollama/myenv/lib/python3.13/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [61]:
def format_docs(docs):
  return "\n".join([doc.page_content for doc in docs])


In [67]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
  {"context": retriever , "question": RunnablePassthrough()}
  | promt
  | llm
  | StrOutputParser()
)

'1. LangGraph is a module built on top of LangChain, designed to facilitate the creation of cyclical graphs, which are often needed for agent runtimes.\n\n2. It provides an easy way to create and manage state graphs, where each graph consists of nodes that update the central state object over time.\n\n3. The state can be updated by either overriding or adding to its value, depending on the use case. For example, if an attribute is a list of actions taken, new actions can be added automatically.'

In [ ]:
rag_chain.invoke({"question": "What is langgraph?"})

In [68]:
rag_chain.invoke({"question": "What is langgraph use for?"})

'1. LangGraph is a module built on top of LangChain, primarily used for creating cyclical graphs, which are often needed for agent runtimes.\n2. It exposes a state interface that can be updated over time by nodes in the graph, and attributes of this state can be either overridden or added to.\n3. Some common use cases include implementing custom and powerful agent runtimes, stateful tools, and multi-agent workflows.'